Since signal dropout and field of view differs slightly across subjects, and since we're planning to analyze voxel data across subjects, we need to first determine the voxels where we have data from enough subjects to run those analyses. That's why here we will compute the number of participants for whom we have data, per voxel. We'll then use the 80% brain mask (i.e. voxels where 80% or more of participants have data) as the mask of interest for each fMRI analysis.

In [ ]:
import sys, os, glob, nltools, scipy, nilearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltools.data import Brain_Data
from nltools.plotting import plot_brain
from nilearn.plotting import plot_img, plot_stat_map, plot_roi

## Load brain masks for video 3

In [ ]:
subs = [2,4,5,6,7,8,10,11,12,13,14,15,16,17,18,19,20,
        21,22,23,24,28,30,31,32,33,35,36,37,38,43,44,45,46,47,48,49,50,51,52,53,54,55]

In [ ]:
data_dir = "/gpfs_home/jvanbaar/data/jvanbaar/polarization/derivatives/fmriprep"

In [ ]:
run = 3

In [ ]:
import nibabel as nib

In [ ]:
sum_mask_arr = np.zeros([65,77,65])
for si,sub in enumerate(subs):
    fname = (data_dir + '/sub-%03d/ses-1/func/'%sub + 
             'sub-%03d_ses-1_task-videoWatching_run-%i'%(sub,run) +
             '_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz')
    mask_img = nib.load(fname)
    mask_arr = mask_img.get_data()
    sum_mask_arr += mask_arr

In [ ]:
sum_mask_img = nib.Nifti1Image(sum_mask_arr, mask_img.affine)

In [ ]:
plot_stat_map(sum_mask_img, display_mode = 'z', cut_coords = np.arange(-40,61,20), cmap = 'viridis')

In [ ]:
sum_mask_img.to_filename('Summed_brain_mask.nii.gz')

## Create threshold masks

In [ ]:
# 100% mask
thresh_mask_arr = np.copy(sum_mask_img.get_data())
thresh_mask_arr[thresh_mask_arr < 43] = 0
thresh_mask_arr[thresh_mask_arr > 0] = 1
thresh_mask_img = nib.Nifti1Image(thresh_mask_arr, sum_mask_img.affine)
plot_stat_map(thresh_mask_img, display_mode = 'z', cut_coords = np.arange(-40,61,20), cmap = 'viridis')
thresh_mask_img.to_filename('Conjunction_brain_mask.nii.gz')

In [ ]:
# 80% mask
thresh_mask_arr = np.copy(sum_mask_img.get_data())
thresh_mask_arr[thresh_mask_arr < 0.8*43] = 0
thresh_mask_arr[thresh_mask_arr > 0] = 1
thresh_mask_img = nib.Nifti1Image(thresh_mask_arr, sum_mask_img.affine)
plot_stat_map(thresh_mask_img, display_mode = 'z', cut_coords = np.arange(-40,61,20), cmap = 'viridis')
thresh_mask_img.to_filename('80pct_brain_mask.nii.gz')

In [ ]:
np.sum(thresh_mask_arr)

In [ ]:
71355/1000